In [1]:
###################################### Prepare data for training Model #####################################

In [2]:
import pyspark
from nltk.tokenize import word_tokenize

Lreviewdata_full = spark.read.parquet("/mnt/etl/parquet/labeled/Lreviewdata")

In [3]:
Lreviewdata = Lreviewdata_full.select("Text","label")
Lreviewdata.registerTempTable("DF")
Lreviewdata.cache()
rdataset = spark.createDataFrame(Lreviewdata.rdd.repartition(8))

sqlContext.clearCache()
rdataset.cache()

Out[ 4 ]: DataFrame[Text: string, label: bigint]

In [4]:
Lreviewdata_full.show(3)

+---+--------------------+-----------+----------+--------------------+------+-----+----------+-------+
key| reviewername|productname|Reviewdate| Text|rating|label|sourcetype|runtype|
+---+--------------------+-----------+----------+--------------------+------+-----+----------+-------+
 1|C0JGKoJXlN36XZXGS...| Snack|2014-11-16|Drinks were bad, ...| 5.0| 1| yelp| Batch|
 2|XcHES9BBbRKDyXN_-...| Snack|2014-09-08|This was the wors...| 5.0| 1| yelp| Batch|
 3|xEt--zB8-4v_31mxk...| Snack|2013-10-06|This is located o...| 5.0| 1| yelp| Batch|
+---+--------------------+-----------+----------+--------------------+------+-----+----------+-------+
only showing top 3 rows

In [5]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

tokenizer = Tokenizer(inputCol="Text",outputCol="tokens")
remover = StopWordsRemover(inputCol="tokens",outputCol="stp_tokens")
pipeline = Pipeline(stages=[tokenizer,remover])

In [6]:
pipe_model = pipeline.fit(rdataset)
tok_dataset = pipe_model.transform(rdataset)

In [7]:
tok_dataset.cache()

Out[ 8 ]: DataFrame[Text: string, label: bigint, tokens: array<string>, stp_tokens: array<string>]

In [8]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')
nltk.download('wordnet')
from nltk.tag import pos_tag
from nltk.corpus import sentiwordnet as swn

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data] /root/nltk_data...
[nltk_data] Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data] Unzipping corpora/sentiwordnet.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Unzipping corpora/wordnet.zip.

In [9]:
tok_dataset.show(1)

+--------------------+-----+--------------------+--------------------+
 Text|label| tokens| stp_tokens|
+--------------------+-----+--------------------+--------------------+
Really didn't see...| 1|[really, didn't, ...|[really, see, veg...|
+--------------------+-----+--------------------+--------------------+
only showing top 1 row

In [10]:
def get_score(word):
  syn = swn.senti_synsets(word)
  for s in syn:
    if s.pos_score() > 0:
      return s.pos_score()
    
def custom_sum(list):
  sum = 0.0
  i = 0
  for j in list:
    if j != None:
      i += 1
      sum += j
  return sum/i if i>0 else 0

In [11]:
tok_df = tok_dataset.select("tokens").rdd.map(lambda x: [custom_sum([get_score(j) for j in i]) for i in x]).toDF()

In [12]:
tok_df.show()

+-------------------+
 _1|
+-------------------+
 0.28125|
 0.125|
0.25961538461538464|
 0.28125|
 0.3048780487804878|
+-------------------+
only showing top 5 rows

In [13]:
from pyspark.sql import functions as f
rate_D_df = tok_df.withColumn('Sentiment', f.when(f.col('_1') > 0.5, '1').when(f.col('_1') == 0.5, '0').otherwise('-1'))

In [14]:
rate_D_df.count()

Out[ 138 ]: 614452

In [15]:
tok_dataset.count()

Out[ 139 ]: 614452

In [16]:
from pyspark.sql.functions import monotonically_increasing_id
rate_D_df = rate_D_df.withColumn('id', monotonically_increasing_id())
tok_dataset = tok_dataset.withColumn("id", monotonically_increasing_id())
df = rate_D_df.join(tok_dataset, "id", "inner").select("Sentiment","Text","label")

In [17]:
df.show()

+---------+--------------------+-----+
Sentiment| Text|label|
+---------+--------------------+-----+
 -1|Ordered delivery ...| 1|
 -1|I never write any...| 1|
 -1|If you don't like...| 1|
 -1|This is one of my...| 1|
 -1|Whether it's the ...| 1|
 -1|This small bodega...| 0|
 -1|Amazing food!!!! ...| 0|
 -1|The best pizza I ...| 1|
 -1|I came here about...| 0|
 -1|If you are in the...| 1|
 -1|Five star blueber...| 1|
 -1|Delicious! Po boy...| 0|
 -1|When in Philly......| 1|
 -1|I've lived on 2nd...| 1|
 -1|I've been here at...| 1|
 -1|I can only echo t...| 1|
 -1|For this review, ...| 1|
 -1|I have been looki...| 1|
 -1|My first impressi...| 1|
 -1|So thrilled that ...| 1|
+---------+--------------------+-----+
only showing top 20 rows

In [18]:
df.write.parquet("/mnt/etl/parquet/labeled/Lreviewdata/Lreviewdata_sentiment/sentiment_analysis.parquet", mode='overwrite')

In [19]:
df.rdd.isEmpty()

Out[ 22 ]: False

In [20]:
Lreviewdata_full_n = spark.read.parquet("/mnt/etl/parquet/labeled/Lreviewdata/Lreviewdata_sentiment/sentiment_analysis.parquet")

In [21]:
Lreviewdata_full_n.show(20)

+---------+--------------------+-----+
Sentiment| Text|label|
+---------+--------------------+-----+
 -1|Really didn't see...| 1|
 -1| The Toast of town!| 1|
 -1|NOISE! Good food,...| 1|
 -1|Fantastic time at...| 1|
 -1|I would give toas...| 1|
 -1|Solid little tea ...| 1|
 -1|Toast has the pot...| 1|
 -1|AWFUL! Had half a...| 1|
 -1|Finally got to tr...| 1|
 -1|I love this place...| 1|
 -1|Ordered pick up 2...| 1|
 -1|U have to be drun...| 1|
 1|Delicious Wings! ...| 1|
 -1|This is more of a...| 1|
 -1|I went there to g...| 1|
 -1|This place used t...| 1|
 -1|Pizza use to be p...| 1|
 -1|This place has th...| 1|
 -1|worst  pizza I ev...| 1|
 -1|It's a good place...| 1|
+---------+--------------------+-----+
only showing top 20 rows